In [1]:
import io
import arcpy
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.widgets import MapView
from zipfile import ZipFile
import os
from osgeo import ogr
from arcgis.geometry import SpatialReference
import sys
from arcpy.sa import *
import arcpy
import numpy as np
from arcpy import env

In [2]:
from arcgis import GIS
gis = GIS()

In [3]:
#access transit stop dataset via api
transitstops_link = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/trans_transit_stops/shp_trans_transit_stops.zip'

stops_response = requests.get(transitstops_link, allow_redirects=True)

In [4]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\shp_trans_transit_stops.zip", 'wb').write(stops_response.content)

2412816

In [5]:
#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\shp_trans_transit_stops.zip", 'r') as z:
    z.printdir()
    
    print('Extracting all files now..')
    z.extractall(r"C:\Users\bende287\Downloads\shp_trans_transit_stops")
    print('Done!')

File Name                                             Modified             Size
STOP_RouteListByStop.cpg                       2024-11-25 00:12:36            5
STOP_RouteListByStop.dbf                       2024-11-25 00:12:36       372546
STOP_RouteListByStop.dbf.xml                   2024-11-25 00:12:36         5286
TransitStops.cpg                               2024-11-25 00:12:38            5
TransitStops.dbf                               2024-11-25 00:12:38     23384594
TransitStops.prj                               2024-11-25 00:12:36          424
TransitStops.sbn                               2024-11-25 00:12:38       209348
TransitStops.sbx                               2024-11-25 00:12:38         5948
TransitStops.shp                               2024-11-25 00:12:38       608036
TransitStops.shp.xml                           2024-11-25 00:12:38        22609
TransitStops.shx                               2024-11-25 00:12:38       173796
metadata/metadata.html                  

In [6]:
# make it a spatially enabled df
df_stops = pd.DataFrame.spatial.from_featureclass(r"C:\Users\bende287\Downloads\shp_trans_transit_stops\TransitStops.shp")
#transform the projection 
df_stops.spatial.project(SpatialReference(4326))

df_stops

,FID,site_id,site_on,site_at,corn_desc,CornerNumb,descriptio,city,county,publiccomm,dt_zone,parkride,adaaccess,busstop_yn,node_id,gate,T_Code,UNIQUE_ID,board_flag,alight_fla,active_sta,itinerary_,IsSuspende,route_prov,SHAPE
0,0,34,Chicago Ave S,8th St S,Farside Southbound,5,Chicago Ave S & 8th St S,MINNEAPOLIS,Hennepin,,Y,N,Y,Y,8SCH,,,7A8DD863-8277-4ADC-A0F2-62D491721677,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26241825103301, ""y"": 44.971087007053..."
1,1,36,Chicago Ave S,14th St E,Nearside Southbound,8,Chicago Ave S & 14th St E,MINNEAPOLIS,Hennepin,,N,N,Y,N,14CH,,,6CBC5983-02F5-4CD8-9FB8-607496F5CF0D,0,0,Archive,N,N,,"{""x"": -93.26262425115756, ""y"": 44.969031007423..."
2,2,37,Chicago Ave S,15th St E,Nearside Southbound,8,Chicago Ave S & 15th St E,MINNEAPOLIS,Hennepin,,N,N,Y,Y,,,,CBBD7A82-964E-412C-8F30-744EFBDCA66E,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26262625100854, ""y"": 44.968179008053..."
3,3,39,Chicago Ave S,17th St E,Nearside Southbound,8,Chicago Ave S & 17th St E,MINNEAPOLIS,Hennepin,,N,N,Y,Y,,,,5CB2F36C-C1A1-4FDE-82FC-581DB8D9D6D5,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26263325102339, ""y"": 44.966415008336..."
4,4,40,Chicago Ave S,18th St E,Farside Southbound,5,Chicago Ave S & 18th St E,MINNEAPOLIS,Hennepin,,N,N,Y,Y,,,,BDBC2BB1-1C23-46EC-B0A3-A502D94BC719,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26263025150253, ""y"": 44.964937008752..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21707,21707,90371,Highway 10,Approaching Big Lake Station,Nearside Eastbound,6,Hwy 10 and County Rd 43 Control Pt,BIG LAKE,Sherburne,Control point - no stop,N,N,N,N,,,,D42980B4-4D80-4D87-B5FA-F1AFECB359DD,0,0,Active,Y,N,St Cloud Metro Bus,"{""x"": -93.73189806625787, ""y"": 45.332247987832..."
21708,21708,90372,171st Avenue Northwest,Approaching Elk River Station,Farside Eastbound,3,171st Ave NW and Hwy 10 Control Pt,ELK RIVER,Sherburne,Control point - no stop,N,N,N,N,,,,136A6CC5-488B-4E52-9963-E76A88A4FC9D,0,0,Active,N,N,,"{""x"": -93.54887112600264, ""y"": 45.281447982245..."
21709,21709,90373,Interstate 394,Approaching 12th Street - Downtown Minneapolis,Midblock Eastbound,10,I-394 to 12th St Exit Control Pt,MINNEAPOLIS,Hennepin,Control point - no stop,N,N,N,N,,,,CC52E289-5113-41EE-A5D3-20712CE58D45,0,0,Active,Y,N,Metro Transit,"{""x"": -93.28663524364403, ""y"": 44.974954008649..."
21710,21710,90374,Century Ave and I-94 Exit Control Point,Century Ave and I-94 Exit Control Point,Nearside Eastbound,6,Century Ave and I-94 Exit Control Pt,MAPLEWOOD,Ramsey,Control point - no stop,N,N,N,N,,,,5351E352-7480-4844-A09B-127AC4E83F09,0,0,Active,N,N,,"{""x"": -92.9857763328953, ""y"": 44.9476089920752..."


In [7]:
#access hennepin county boundary to clip other layers to 
hennepinboundary_link = r"https://gis.hennepin.us/arcgis/rest/services/HennepinData/BOUNDARIES/MapServer/1/query?where=1%3D1&f=json"

bdry_response = requests.get(hennepinboundary_link, stream=True).json()

In [8]:
type(bdry_response)

<class 'dict'>

In [9]:
#going from dictionary to feature set here
fs = FeatureSet.from_dict(bdry_response)

bdry_fs = fs.sdf
bdry_fs

#making sure to maintain the same projection throughout-- transforming to WGS 84
#arc_rest.spatial.project(SpatialReference(4326))
#arc_rest

,NAME_TXT,OBJECTID,SHAPE
0,HENNEPIN COUNTY,1,"{""rings"": [[[459137.16889999993, 5010412.97660..."


In [10]:
#bike and pedestrian system layer
bikeandpeds_link = r"https://gis.hennepin.us/arcgis/rest/services/HennepinData/TRANSPORTATION/MapServer/1/query?where=1%3D1&f=json"

bikeandpeds_response = requests.get(bikeandpeds_link, stream=True).json()

type(bikeandpeds_response)

<class 'dict'>

In [11]:
#going from dictionary to feature set here
fs2 = FeatureSet.from_dict(bikeandpeds_response)

bikeandpeds_fs = fs2.sdf
bikeandpeds_fs


,TRAILNAME,OBJECTID,SHAPE
0,<NA>,1,"{""paths"": [[[458854.48649999965, 4963932.01380..."
1,<NA>,2,"{""paths"": [[[458886.23950000014, 4963936.6953]..."
2,<NA>,3,"{""paths"": [[[458923.60780000035, 4963943.7355]..."
3,<NA>,4,"{""paths"": [[[459306.29590000026, 4964213.52899..."
4,<NA>,5,"{""paths"": [[[472044.51159999985, 4961658.24640..."
...,...,...,...
1995,<NA>,1996,"{""paths"": [[[457367.9462000001, 5000558.634299..."
1996,<NA>,1997,"{""paths"": [[[457358.34300000034, 5000067.23740..."
1997,<NA>,1998,"{""paths"": [[[457394.80229999963, 5002058.5033]..."
1998,<NA>,1999,"{""paths"": [[[457392.1763000004, 5001612.020400..."


In [12]:
#Parcel dataset that will eventually be joined to the land for sale data.
parcel_link = r'https://gis.hennepin.us/arcgis/rest/services/HennepinData/LAND_PROPERTY/MapServer/1/query?where=1%3D1%20AND%201%3D1&f=json'

parcel_response = requests.get(parcel_link, stream=True).json()

type(parcel_response)

<class 'dict'>

In [13]:
#going from dictionary to feature set here
fs3 = FeatureSet.from_dict(parcel_response)

parcel_fs = fs3.sdf
parcel_fs


,SALE_CODE_NAME,OBJECTID,SHAPE
0,WARRANTY DEED,1,"{""rings"": [[[481758.6118999999, 4967622.9012],..."
1,,2,"{""rings"": [[[481827.7642000001, 4967621.761100..."
2,EXCLUDED FROM RATIO STUDIES,3,"{""rings"": [[[481827.44710000046, 4967582.15430..."
3,,4,"{""rings"": [[[481799.1160000013, 4967582.621400..."
4,,5,"{""rings"": [[[482035.27010000125, 4967618.3399]..."
...,...,...,...
1995,WARRANTY DEED,1996,"{""rings"": [[[482004.89219999965, 4976424.7015]..."
1996,OTHER – SEE CERTIFICATE OF REAL ESTATE VALUE (...,1997,"{""rings"": [[[482010.4545000009, 4976413.867699..."
1997,WARRANTY DEED,1998,"{""rings"": [[[481912.8088999996, 4976443.9617],..."
1998,,1999,"{""rings"": [[[481892.46570000146, 4976433.5561]..."


In [14]:
parks_URL = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/bdry_metro_colabtiv_parks/shp_bdry_metro_colabtiv_parks.zip'

parks_response = requests.get(parks_URL, allow_redirects=True)

In [15]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\shp_bdry_metro_colabtiv_parks.zip", 'wb').write(parks_response.content)

5525043

In [16]:
#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\shp_bdry_metro_colabtiv_parks.zip", 'r') as c:
    c.printdir()
    
    print('Extracting all files now..')
    c.extractall(r"C:\Users\bende287\Downloads\shp_bdry_metro_colabtiv_parks")
    print('Done!')

File Name                                             Modified             Size
MetroCollaborativeParks.cpg                    2024-09-23 17:59:50            5
MetroCollaborativeParks.dbf                    2024-09-23 17:59:50     12172519
MetroCollaborativeParks.prj                    2024-09-23 17:59:50          424
MetroCollaborativeParks.sbn                    2024-09-23 17:59:50        37308
MetroCollaborativeParks.sbx                    2024-09-23 17:59:50         2404
MetroCollaborativeParks.shp                    2024-09-23 17:59:50      8507476
MetroCollaborativeParks.shp.xml                2024-09-23 17:59:50        77943
MetroCollaborativeParks.shx                    2024-09-23 17:59:50        31020
metadata/metadata.html                         2024-08-06 16:11:02        13462
metadata/metadata.xml                          2024-08-06 16:11:02         6542
Extracting all files now..
Done!


In [17]:
# make it a spatially enabled df
df_parks = pd.DataFrame.spatial.from_featureclass(r"C:\Users\bende287\Downloads\shp_bdry_metro_colabtiv_parks\MetroCollaborativeParks.shp")
#transform the projection 
#df_parks.spatial.project(SpatialReference(4326))

df_parks

,FID,PARKID,PARKNAME,NAME_ALT,PARK_ADDR,CITY_MUNI,COUNTY,STATE,PARKSTATUS,WINTER_STS,FUNC_TYPE,LANDOWNER,OWNERTYPE,AGENCYNAME,AGENCYTYPE,YEAROPEN,LANDCOVER,WATERACRES,LAND_ACRES,PARK_ACRES,PLND_ACRES,TRL_MILES,ACQSOURCE,ACQMETHOD,ACQCOMMENT,DEVSOURCE,DEVCOMMENT,RESTRICTS,LWCFPROT,OTHERPROT,RESPRTCMNT,SERVICAREA,MGMT_PRIOR,VLTR_OPPS,SPEC_FEAT,PARK_HOURS,PKING_CAP,VISITORCTS,PASSHOLDER,PARK_EMAIL,PARK_PHONE,PARK_URL,MNTCEEMAIL,MNTCEPHONE,DATAAUTHOR,EDITED_BY,EDITED_DT,CREATED_BY,CREATED_DT,Shape_Leng,Shape_Area,SHAPE
0,0,1,Islands of Peace Park,,200 Charles St NE,Fridley,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,70.035095,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.fridley.mn.us/DocumentCenter/Vie...,,763-571-3450,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,4937.894919,283421.975373,"{""rings"": [[[477871.7566999998, 4991062.148499..."
1,1,2,Coon Lake County Park,,5450 197th Ave NE,Columbus,003,MN,Open,Open,County Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,111.440554,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,https://www.anokacounty.us/722/Coon-Lake-Count...,,763-324-3300,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,3681.878922,450983.919852,"{""rings"": [[[490172.79389999993, 5019062.42809..."
2,2,3,Manomin Park,,6666 East River Rd,Fridley,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,13.766334,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.fridley.mn.us/DocumentCenter/Vie...,,763-571-3450,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,1204.812861,55710.376642,"{""rings"": [[[478289.28330000024, 4993075.01789..."
3,3,4,Riverfront Regional Park,,5100 East River Rd,Fridley,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,66.14842,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.fridley.mn.us/DocumentCenter/Vie...,,763-571-3450,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,4581.75324,267693.160293,"{""rings"": [[[477894.19570000004, 4990291.52270..."
4,4,5,Lake George Regional Park,,2859 Lake George Dr NW,Oak Grove,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,257.882298,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.oak-grove.mn.us/vertical/sites/%...,,763-404-7000,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,5218.489393,1043612.6344,"{""rings"": [[[474097.65950000007, 5023290.9901]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3860,3860,163-3821,Lake Elmo Park Reserve,,1515 KEATS AVE N,Lake Elmo,163,MN,Open Fee,Open,,Washington County,03,Washington County,03,0,,0.0,0.0,2177.03488,0,0.0,,,,,,,,,,,,,Swim Pond,6 A.M. - 10 P.M.,0,0,0,parks@co.washington.mn.us,651-430-8370,https://www.co.washington.mn.us/502/Lake-Elmo-...,,,Washington County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,381.50367,8085.588943,"{""rings"": [[[509222.81599999964, 4979310.0371]..."
3861,3861,163-3822,Oak Park Crossing Park,,N/A,Oak Park Heights,163,MN,Open,Unknown,,XCEL Energy,99,Oak Park Heights,05,2015,,0.0,0.0,35.0,0,0.0,,,,,,,,,,,,,,,0,0,0,,,,,,Oak Park Heights,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,1855.172978,138922.342238,"{""rings"": [[[516126.4826999996, 4986830.0363],..."
3862,3862,163-3824,Autumn Hills Park,,N/A,Oak Park Heights,163,MN,Open,Unknown,,VALLEY SENIOR SERV ALLIANCE,99,Oak Park Heights,05,2007,,0.0,0.0,0.0,0,0.0,,,,,,,,,,,,,,,0,0,0,,,,,,Oak Park Heights,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,1064.730402,73824.970108,"{""rings"": [[[513179.2923999997, 4986383.2434],..."
3863,3863,163-3825,Linear Park,,N/A,Oak Park Heights,163,MN,Open,Unknown,,Oak Park Heights,05,Oak Park Heights,05,0,,0.0,0.0,0.0,0,0.0,,,,,,,,,,,,,,

In [18]:
landcover_link = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/edu_umn/base_landcover_minnesota/tif_base_landcover_minnesota.zip'

lc_response = requests.get(landcover_link, allow_redirects=True)
     

In [19]:

#open a new file and use the dot interface. write to put data in the file
open(r'C:\Users\bende287\Downloads\tif_base_landcover_minnesota.zip', 'wb').write(lc_response.content)
     


97106583

In [20]:
#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\tif_base_landcover_minnesota.zip", 'r') as g:
    g.printdir()
    
    print('Extracting all files now..')
    g.extractall(r"C:\Users\bende287\Downloads\tif_base_landcover_minnesota")
    print('Done!')

File Name                                             Modified             Size
landcover_impervious_statewide2013_v2.tif      2016-07-21 11:21:44   1691043568
metadata/metadata.html                         2016-10-21 11:43:38        31278
metadata/metadata.xml                          2016-10-25 10:58:54        17434
metadata/preview.jpg                           2016-10-21 11:43:38       320230
metadata/Thumbs.db                             2016-11-22 13:50:36        22528
Extracting all files now..
Done!


In [ ]:
#Don't know if I will use this dataset.. don't run!
equity_URL = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/society_equity_considerations/shp_society_equity_considerations.zip'

equity_response = requests.get(equity_URL, allow_redirects=True)

#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\shp_society_equity_considerations.zip", 'wb').write(equity_response.content)

#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\shp_society_equity_considerations.zip", 'r') as e:
    e.printdir()
    
    print('Extracting all files now..')
    e.extractall(r"C:\Users\bende287\Downloads\shp_society_equity_considerations")
    print('Done!')
    
# make it a spatially enabled df
df_equity = pd.DataFrame.spatial.from_featureclass(r"C:\Users\bende287\Downloads\shp_society_equity_considerations\EquityConsiderationsACPACACurrent.shp")
#transform the projection 
#df_parks.spatial.project(SpatialReference(4326))

df_equity

In [22]:
#Next cells are going to clip my layers 

In [21]:
arcpy.analysis.Clip(
    in_features="TransitStops",
    clip_features="Hennepin_County_Boundary",
    out_feature_class=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\TransitStops_Clip",
    cluster_tolerance=None
)

arcpy.analysis.Clip(
    in_features="MetroCollaborativeParks",
    clip_features="Hennepin_County_Boundary",
    out_feature_class=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\MetroCollaborativeParks_Clip",
    cluster_tolerance=None
)


arcpy.management.Clip(
    in_raster="landcover_impervious_statewide2013_v2.tif",
    rectangle="439356.3565 4959127.3545 486031.7393 5010414.0576",
    out_raster=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\landcover_impervious_Clip",
    in_template_dataset="Hennepin_County_Boundary",
    nodata_value="255",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\GIS5571FinalProj\\GIS5571FinalProj.gdb\\landcover_impervious_Clip'>

In [22]:
#this tool modifies the input features (County_Parcels)-- check attribute table
#takes a few minutes to run
arcpy.analysis.Near(
    in_features="County_Parcels",
    near_features="TransitStops_Clip",
    search_radius=None,
    location="NO_LOCATION",
    angle="NO_ANGLE",
    method="PLANAR",
    field_names="NEAR_FID NEAR_SID;NEAR_DIST NEAR_Stops",
    distance_unit="Kilometers"
)

arcpy.analysis.Near(
    in_features="County_Parcels",
    near_features="Hennepin_County_Bike_and_Pedestrian_System",
    search_radius=None,
    location="NO_LOCATION",
    angle="NO_ANGLE",
    method="PLANAR",
    field_names="NEAR_FID NEAR_BID;NEAR_DIST NEAR_Bike",
    distance_unit="Kilometers"
)

arcpy.analysis.Near(
    in_features="County_Parcels",
    near_features="MetroCollaborativeParks_Clip",
    search_radius=None,
    location="NO_LOCATION",
    angle="NO_ANGLE",
    method="PLANAR",
    field_names="NEAR_FID NEAR_PID;NEAR_DIST NEAR_Park",
    distance_unit="Kilometers"
)

<Result 'County_Parcels'>

In [23]:
#reclassify landcover layer--see metadata for remap values
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="landcover_impervious_Clip",
        reclass_field="Value",
        remap=" 0 101 3; 101 104 4; 104 108 1; 109 111 2",
        missing_values="DATA"
    )
    out_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\Reclass_landcover")
#comes out as "out_raster"

In [24]:
#Convert parcel to raster --raster reclassification

#Jenks algorithm will find natural groupings that exist in the data
#takes a few minutes to run
##ASC = class with lowest values assigned to 1

arcpy.management.ReclassifyField(
    in_table="County_Parcels",
    field="NEAR_Bike",
    method="NATURAL_BREAKS",
    classes=4,
    interval=None,
    standard_deviations="ONE",
    reclass_table=None,
    reverse_values="ASC",
    output_field_name="REBike"
)

arcpy.management.ReclassifyField(
    in_table="County_Parcels",
    field="NEAR_Stops",
    method="NATURAL_BREAKS",
    classes=4,
    interval=None,
    standard_deviations="ONE",
    reclass_table=None,
    reverse_values="ASC",
    output_field_name="REStop"
)

#DESC = class with highest values assigned to 1
arcpy.management.ReclassifyField(
    in_table="County_Parcels",
    field="NEAR_Park",
    method="NATURAL_BREAKS",
    classes=4,
    interval=None,
    standard_deviations="ONE",
    reclass_table=None,
    reverse_values="DESC",
    output_field_name="REPark"
)



<Result 'County_Parcels'>

In [25]:

arcpy.conversion.FeatureToRaster(
    in_features="County_Parcels_filteredMKT_VAL",
    field="REBike",
    out_raster=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\Feature_Bike",
    cell_size=179.291768800002
)


<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\GIS5571FinalProj\\GIS5571FinalProj.gdb\\Feature_Bike'>

In [26]:
arcpy.conversion.FeatureToRaster(
    in_features="County_Parcels_filteredMKT_VAL",
    field="REPark",
    out_raster=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\Feature_Park",
    cell_size=179.291768800002
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\GIS5571FinalProj\\GIS5571FinalProj.gdb\\Feature_Park'>

In [27]:
arcpy.conversion.FeatureToRaster(
    in_features="County_Parcels_filteredMKT_VAL",
    field="REStop",
    out_raster=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\Feature_Stop",
    cell_size=179.291768800002
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\GIS5571FinalProj\\GIS5571FinalProj.gdb\\Feature_Stop'>

In [28]:
#here i am assigning weights to the layers according to influence that they should have on the model
weightedlandcover = Raster("out_raster")*0.25

weightedbikepaths = Raster("Feature_Bike")*0.33

weightedparks = Raster("Feature_Park")*0.09
     
weightedStops = Raster("Feature_Stop")*0.33


In [29]:
#using map algebra to combine our weighted datasets and create a cost surface to input into optimal region connections
combined_datasets = Raster("weightedbikepaths") + Raster("weightedlandcover") + Raster("weightedparks") + Raster("weightedStops")

In [30]:
#Used to generate a random weighting scheme for sens
n = 4

weight_list = np.random.dirichlet(np.ones(n), size=1)[0].tolist()

In [31]:
print(weight_list)

[0.6570666384536252, 0.13978915743887158, 0.021551282609164245, 0.18159292149833905]


In [32]:
#here i am assigning weights to the layers according to influence that they should have on the model
weightedlandcover1 = Raster("out_raster")*0.6570666384536252

weightedbikepaths1 = Raster("Feature_Bike")*0.13978915743887158

weightedparks1 = Raster("Feature_Park")*0.021551282609164245
     
weightedStops1 = Raster("Feature_Stop")*0.18159292149833905
